In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random as rand

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

2023-04-18 14:17:35.832010: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 14:17:36.367290: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
os.chdir('../../')

In [3]:
from programs import lstm_tool

2023-04-18 14:17:37.454022: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
# 예측값이 맞는지 아닌지 판단하는 모델을 만들 거야
# mse는 label이 있어야 하는데 이거는 학습만 시키면 그게 필요 없지
# GAN이라고 봐도 상관 없고

In [5]:
df = pd.read_excel('./documents/nov_nine_var.xlsx').iloc[:,1::].to_numpy()

scaler = MinMaxScaler()
scale_df = scaler.fit_transform(df[:,0:21])

In [7]:
sequence, state = lstm_tool.make_sequence(scale_df, 60)

lable = np.concatenate((np.ones(state.shape[0]), np.ones(state.shape[0]), np.zeros(state.shape[0]), np.zeros(state.shape[0]), np.zeros(state.shape[0])))

sequence = np.concatenate((sequence, sequence, sequence, sequence, sequence))

state = np.concatenate((state, state, np.random.permutation(state), np.random.permutation(state), np.random.permutation(state)))

In [8]:
sequence.shape

(1850, 60, 21)

In [9]:
sequence_train, sequence_test, state_train, state_test, lable_train, lable_test = train_test_split(sequence, state, lable, test_size=0.3, shuffle=True)

In [10]:
lstm_input = tf.keras.layers.Input(shape=(60, 21))

input_lstm_layer = tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=True)(lstm_input)
hidden_lstm_layer0 = tf.keras.layers.LSTM(256, activation='tanh', kernel_initializer=tf.keras.initializers.GlorotUniform(), return_sequences=False)(input_lstm_layer)
dropout_lstm_layer = tf.keras.layers.Dropout(0.2)(hidden_lstm_layer0)
hidden_lstm_layer1 = tf.keras.layers.Dense(128, activation='relu')(dropout_lstm_layer)
hidden_lstm_layer2 = tf.keras.layers.Dense(128, activation='relu')(hidden_lstm_layer1)

lstm_layer = tf.keras.models.Model(lstm_input, hidden_lstm_layer2)

state_input = tf.keras.layers.Input(shape=(21,))

input_state_layer = tf.keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(state_input)
dropout_state_layer = tf.keras.layers.Dropout(0.3)(input_state_layer)
hidden_state_layer0 = tf.keras.layers.Dense(64, activation='relu')(dropout_state_layer)
hidden_state_layer1 = tf.keras.layers.Dense(64, activation='relu')(hidden_state_layer0)

state_layer = tf.keras.models.Model(state_input, hidden_state_layer1)

input_concat_layer = tf.keras.layers.concatenate([lstm_layer.output, state_layer.output])
hidden_concat_layer0 = tf.keras.layers.Dense(32, activation='relu')(input_concat_layer)
dropout_concat_layer = tf.keras.layers.Dropout(0.3)(hidden_concat_layer0)
hidden_concat_layer1 = tf.keras.layers.Dense(32, activation='relu')(dropout_concat_layer)
output_concat_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_concat_layer1)

model = tf.keras.models.Model([lstm_input, state_input], output_concat_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

2023-04-18 14:17:53.371091: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-18 14:17:53.371896: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-18 14:17:53.372491: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [11]:
model.fit([sequence_train, state_train], lable_train, epochs=1000, batch_size=1024, 
          validation_data=([sequence_test, state_test], lable_test), verbose=1)
# , callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)])

Epoch 1/1000


2023-04-18 14:17:54.768629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-18 14:17:54.769509: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-18 14:17:54.770160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - ETA: 0s - loss: 0.7222 - accuracy: 0.3992

2023-04-18 14:17:58.525299: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-18 14:17:58.526129: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-18 14:17:58.526747: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 4s 1s/step - loss: 0.7222 - accuracy: 0.3992 - val_loss: 0.6926 - val_accuracy: 0.5261
Epoch 2/1000
2/2 [==============================] - 2s 567ms/step - loss: 0.6977 - accuracy: 0.4371 - val_loss: 0.6844 - val_accuracy: 0.6000
Epoch 3/1000
2/2 [==============================] - 2s 568ms/step - loss: 0.6877 - accuracy: 0.5722 - val_loss: 0.6758 - val_accuracy: 0.6000
Epoch 4/1000
2/2 [==============================] - 2s 558ms/step - loss: 0.6782 - accuracy: 0.5876 - val_loss: 0.6739 - val_accuracy: 0.6000
Epoch 5/1000
2/2 [==============================] - 2s 561ms/step - loss: 0.6763 - accuracy: 0.5846 - val_loss: 0.6740 - val_accuracy: 0.6000
Epoch 6/1000
2/2 [==============================] - 2s 567ms/step - loss: 0.6819 - accuracy: 0.5892 - val_loss: 0.6732 - val_accuracy: 0.6000
Epoch 7/1000
2/2 [==============================] - 2s 559ms/step - loss: 0.6733 - accuracy: 0.5977 - val_loss: 0.6734 - val_accuracy: 0.6000
Epoch 8/1000
2/2 [====

In [ ]:
djjakfdlksjf

In [12]:
model.save('./model/checker.h5')